# Pytorch Introduction


In [ ]:
%pip install torch
import torch

In [ ]:
torch.__version__

In [ ]:
scalar = torch.tensor(7)
vector = torch.tensor([1, 2, 3])
Matrix = torch.tensor([[1, 2, 3], [4, 5, 6]])
T = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [0, 1, 2]]])
T.shape

# Random Tensors

In [ ]:
M1 = torch.rand(3, 3)
M2 = torch.rand(10,10)
T1 = torch.rand(3, 4, 4)
T1

In [ ]:
matrix_ones = torch.ones(5, 5)

# Use torch.triu() to get the upper triangular matrix
triangular_matrix = torch.triu(matrix_ones)

triangular_matrix

In [ ]:
# Use torch.tril() to get the lower triangular matrix
triangular_matrix = torch.tril(matrix_ones)
triangular_matrix

In [ ]:
# from a full random matrix of 5x5 get the first element of row 1, the first two elements of row 2, the first three elements of row 3, and so on
matrix = torch.rand(5, 5)
print(matrix)
result = matrix * triangular_matrix
result


In [ ]:
# create a matrix of ones "like" result, but with the ones on the lower triangular part
# make lower triangle ones by usine torch.like
lower_ones = torch.ones_like(result).tril()
lower_ones







In [ ]:
# matrix multiplication
A = torch.ones(2,3)
M = torch.matmul(A, A.T)
M

In [ ]:
# Squeeze removes all the dimensions that have a size of 1
A = torch.tensor([[1, 2, 3]]).squeeze()
A.shape

In [ ]:
# permute the dimensions of a tensor for a color image 10x10x3 with 3 channels
image = torch.rand(10, 10, 3)
print(image.shape)
new_image = image.permute(2, 0, 1)  
new_image.shape

In [ ]:
# indexing and slicing
# create a 3x3 matrix with random values
matrix = torch.rand(3, 3)
print(matrix)
# get the first row
first_row = matrix[0, 0:2]
first_row

In [ ]:
# Get a GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Python Classes https://www.geeksforgeeks.org/python-classes-and-objects/#
class Dog:
    # Class Attribute
    species = 'mammal'
    # Initializer / Instance Attributes
    def __init__(self, name, age):
        self.name = name
        self.age = age
    # instance method
    def description(self):
        return "{} is {} years old".format(self.name, self.age)
    # instance method
    def speak(self, sound):
        return "{} says {}".format(self.name, sound)
    
# Instantiate the Dog object
mikey = Dog("Mikey", 6)
# call our instance methods
print(mikey.description())
print(mikey.speak("Gruff Gruff"))

# Linear Regression Model

In [ ]:
# generate training data for line y = 2x + 3
import numpy as np
import torch
X_train = torch.tensor([[1, 1], [2, 1], [3, 1], [4, 1], [5, 1]], dtype=torch.float32)
y_train = torch.tensor([[5], [7], [9], [11], [13]], dtype=torch.float32)
print(X_train)
print(y_train)

In [216]:
# Build the Neural Network
class LinearRegressionNet(nn.Module):
    def __init__(self):
        super(LinearRegressionNet, self).__init__()
        # One fully connected layer (input_dim=2, output_dim=1)
        self.fc = nn.Linear(2, 1)
        
    def forward(self, x):
        return self.fc(x)

# Instantiate the network
net = LinearRegressionNet()

# Define the loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Train the Network
num_epochs = 1000
for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = net(x)
    loss = criterion(outputs, y)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    # Print loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the network
test_data = torch.tensor([[6, 1], [7, 1]], dtype=torch.float32)
predictions = net(test_data)
predictions


Epoch [100/1000], Loss: 0.2075
Epoch [200/1000], Loss: 0.0585
Epoch [300/1000], Loss: 0.0165
Epoch [400/1000], Loss: 0.0047
Epoch [500/1000], Loss: 0.0013
Epoch [600/1000], Loss: 0.0004
Epoch [700/1000], Loss: 0.0001
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000


tensor([[15.0024],
        [17.0034]], grad_fn=<AddmmBackward0>)

# Recode using weights as nn.Parameter

In [241]:
import torch.nn as nn
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.weights = nn.Parameter(torch.tensor([1, 1], requires_grad=True, dtype=torch.float32))
        
    def forward(self, x):
        # CHECK THIS FOR CORRECTNESS
        return (x * self.weights).sum(dim=1, keepdim=True)

In [242]:

model_01 = LinearRegressionModel()
list(model_01.parameters())
model_01.state_dict()

OrderedDict([('weights', tensor([1., 1.]))])

# model predictive power use torch.inference_mode()


In [243]:
with torch.inference_mode():
    y_preds =  model_01(X_train)
    print(y_preds)

tensor([[2.],
        [3.],
        [4.],
        [5.],
        [6.]])


# Loss Function
### define the params to be optimized and lr the learning rate

In [244]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(params = model_01.parameters(), lr=0.01)

In [246]:
# training loop
epochs = 1000
for epoch in range(epochs):
    model_01.train()
    y_preds = model_01(X_train)
    loss = loss_fn(y_preds, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Epoch: {}, Loss: {}".format(epoch, loss.item()))
# Test the network
test_data = torch.tensor([[6, 1], [7, 1]], dtype=torch.float32)
predictions = model_01(test_data)
predictions

Epoch: 0, Loss: 0.000533123267814517
Epoch: 1, Loss: 0.0005295381997711957
Epoch: 2, Loss: 0.0005259484169073403
Epoch: 3, Loss: 0.0005224148044362664
Epoch: 4, Loss: 0.0005188776995055377
Epoch: 5, Loss: 0.0005153750535100698
Epoch: 6, Loss: 0.0005118901608511806
Epoch: 7, Loss: 0.0005084351287223399
Epoch: 8, Loss: 0.0005050087347626686
Epoch: 9, Loss: 0.0005015895585529506
Epoch: 10, Loss: 0.0004982058308087289
Epoch: 11, Loss: 0.0004948460264131427
Epoch: 12, Loss: 0.0004915059544146061
Epoch: 13, Loss: 0.00048818904906511307
Epoch: 14, Loss: 0.0004849030519835651
Epoch: 15, Loss: 0.00048161851009353995
Epoch: 16, Loss: 0.0004783787881024182
Epoch: 17, Loss: 0.00047514549805782735
Epoch: 18, Loss: 0.00047193473437801003
Epoch: 19, Loss: 0.0004687482141889632
Epoch: 20, Loss: 0.0004655787197407335
Epoch: 21, Loss: 0.00046244956320151687
Epoch: 22, Loss: 0.0004593218327499926
Epoch: 23, Loss: 0.00045622707693837583
Epoch: 24, Loss: 0.00045314175076782703
Epoch: 25, Loss: 0.0004500736

tensor([[15.0012],
        [17.0017]], grad_fn=<SumBackward1>)